# A* Algorithm
I constructed two classes for this matter. One is `Node`, which is responsible for calculating `f` and `g` and holding each node's parents to be able to find a path at the end.

The second class is `Graph`, which only holds the adjacency matrix and heuristics.

In [1]:
class Node:
	def __init__(self, id_, heuristic):
		self.id = id_

		self.parent = None
		
		self._g = 0
		self._h = heuristic

	@property
	def g(self):
		return self._g

	@property
	def h(self):
		return self._h

	@property
	def f(self):
		return self._f

	@f.setter
	def f(self, f):
		self._f = f

	@g.setter
	def g(self, g):
		self._g = g
		self.f = self.h + self.g

	@h.setter
	def h(self, h):
		self._h = h
		self.f = self.h + self.g

	def set_parent(self, node, distance):
		self.parent = node
		self.g = node.g + distance

	def is_better_parent(self, node, distance):
		g = node.g + distance
		f = self.h + g
		return f < self.f


class Graph:
	def __init__(self, adj, heuristics):
		self.adj = adj
		self.n = len(adj)
		self.heuristics = heuristics


The actual algorithm is implemented below. We have a `used_nodes` dictionary in which collects created nodes so we could find which node has been seen before and update its parent (path to reach it) if it is better.

We decide which node we should visit based on our priority queue, `queue`. It is sorted based on `f` value of each node. `f` is the sum of `g`, the cost of reaching to that node, and `h`, the heuristic (estimated cost of reaching goal from that node).

At first, there is only the starting node in the queue, but as our loop execute, the first element of the queue (the node with the least `f`) gets popped. If the popped element (`q`) is the our target the loop breaks and we return the founded path. Otherwise, we check all of this nodes successors. If a successor has been visited before, we check if reaching it from q is better than how it was reached before and our metric is `f` (actually it is `g` because `h` is constant). If it is better, we change its parent to `q` and add itself to the queue. If a successor has not been checked yet, we create its node and make `q` its parent and add it to the queue.

Finally, we return the last node, the target node, as the its ancestors are the best path to reach it.

In [2]:
def AStar_search(graph):
	used_nodes = dict()
	queue = []

	root = Node(0, graph.heuristics[0])

	queue.append(root)

	while len(queue):
		# find the item with the least value and pop
		q = queue.pop(0)

		if q.id == graph.n - 1:
			return q

		for successor in range(graph.n):
			if graph.adj[q.id][successor] == -1:
				continue

			if successor in used_nodes.keys():
				node = used_nodes[successor]
				if node.is_better_parent(q, graph.adj[q.id][node.id]):
					node.set_parent(q, graph.adj[q.id][node.id])
					if node not in queue:
						queue.append(node)
			else:
				node = Node(successor, graph.heuristics[successor])
				node.set_parent(q, graph.adj[q.id][node.id])
				used_nodes[successor] = node
				queue.append(node)
		
	queue = sorted(queue, key=lambda x: x.f)
	
	return used_nodes[graph.n - 1]


Here we have a function that prints the path based on the target node.

In [3]:
def print_path(node, names):
	if node.parent is None:
		print('Path:', names[node.id], end="")
		return
	print_path(node.parent, names)
	print(",", names[node.id], end="")


Our test case is the below graph.


![Graph Problem 1](problem1.jpg)

In [4]:
adj = [
	[-1, 6, -1, -1, -1, 3, -1, -1, -1, -1],
	[6, -1, 3, 2, -1, -1, -1, -1, -1, -1],
	[-1, 3, -1, 1, 5, -1, -1, -1, -1, -1],
	[-1, 2, 1, -1, 8, -1, -1, -1, -1, -1],
	[-1, -1, 5, 8, -1, -1, -1, -1, 5, 5],
	[3, -1, -1, -1, -1, -1, 1, 7, -1, -1],
	[-1, -1, -1, -1, -1, 1, -1, -1, 3, -1],
	[-1, -1, -1, -1, -1, 7, -1, -1, 2, -1],
	[-1, -1, -1, -1, 5, -1, 3, 2, -1, 3],
	[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
]

heuristics = [10, 8, 5, 7, 3, 6, 5, 3, 1, 0]

names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

graph = Graph(adj, heuristics)



node = AStar_search(graph)
print('Cost:', node.f)
print_path(node, names)



Cost: 10
Path: A, F, G, I, J